In [4]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/Grad/DSAN6600/proj/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Grad/DSAN6600/proj


In [22]:
import os
import glob
import torch
import numpy as np
import pandas as pd
from PIL import Image
from IPython.display import Image, display

# Load Data + YOLO

In [39]:
raw_imgs_path = 'data/raw'
all_imgs = glob.glob(os.path.join(raw_imgs_path, '*.jpg'))

### DOWNSAMPLED FOR TESTING ###
# sample 10
# sampled_imgs = all_imgs[:10]

# # load imgs
# imgs = [Image.open(img_path) for img_path in sampled_imgs]

### THE REAL DEAL ###
# load imgs
imgs = [Image.open(img_path) for img_path in all_imgs]

# load yolo
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-7 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
results = model(imgs)

# sanity check
# results.print()
dets = results.pandas().xyxy

# only people w/ at least 50% confidence
people_det = [det[(det['class'] == 0) & (det['confidence'] >= 0.5)] for det in dets]

In [ ]:
def crop_and_save(img_paths, detections_list, verbose=False):
  out_path = 'data/subjectbox'
  os.makedirs(out_path, exist_ok=True)

  for img_path, detections in zip(img_paths, detections_list):
    # load img
    img = Image.open(img_path)

    for index, row in detections.iterrows():
      # extract bounding box coords; use ints for saving img name
      xmin, ymin, xmax, ymax = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])

      # crop image around bounding box
      cropped_img = img.crop((xmin, ymin, xmax, ymax))

      # get image name; remove extension
      img_name = os.path.basename(img_path)
      img_name_no_ext = os.path.splitext(img_name)[0]

      # create new image name w/ bounding box coords
      new_img_name = f"{img_name_no_ext}_subject_box_{xmin}_{ymin}_{xmax}_{ymax}.jpg"

      # save cropped img
      save_path = os.path.join(out_path, new_img_name)
      cropped_img.save(save_path)

      if verbose:
        print(f"Saved: {save_path}")

In [ ]:
### DOWNSAMPLED FOR TESTING ###
# crop_and_save(sampled_imgs, people_det, verbose=True)

### THE REAL DEAL ###
crop_and_save(all_imgs, people_det, verbose=False)